## SMART TRAFFIC LIGHT CONTROL SYSTEM

In [ ]:
import os
import sys
import importlib
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
import pandas as pd
import time
import traceback

# Set working directory to ensure generate_report.py is found
os.chdir(os.path.dirname(os.path.abspath(__file__))) if '__file__' in globals() else os.chdir(os.getcwd())
print(f"Current working directory: {os.getcwd()}")

# Verify generate_report.py exists
if not os.path.exists('generate_report.py'):
    print("Error: 'generate_report.py' not found in current directory.")
    print("Please ensure 'generate_report.py' is in the same directory as this notebook.")
    raise FileNotFoundError("generate_report.py not found")

# Import generate_report
try:
    import generate_report
    importlib.reload(generate_report)  # Reload to avoid Jupyter caching issues
    print("Successfully imported generate_report module")
except ImportError as e:
    print(f"Error: Failed to import generate_report - {e}")
    print("Please install the required libraries using:")
    print("pip install scikit-fuzzy numpy scipy networkx matplotlib scikit-learn pandas python-docx")
    raise


# Step 1: Generate Realistic Dataset with All Parameters

In [ ]:
def generate_realistic_traffic_data(n_samples=10000):
    np.random.seed(42)
    data = {
        'vehicle_density': [],
        'waiting_time': [],
        'flow_rate': [],
        'queue_length': [],
        'emergency_vehicle': [],
        'opposing_density': [],
        'green_duration': []
    }
    for _ in range(n_samples):
        period = np.random.choice(['rush', 'off-peak', 'night'], p=[0.3, 0.5, 0.2])
        if period == 'rush':
            density = np.random.randint(70, 100)
            wait = np.random.randint(40, 60)
            flow = np.random.randint(40, 60)
            queue = np.random.randint(20, 30)
            emergency = np.random.choice([0, 1], p=[0.95, 0.05])
            opp_density = np.random.randint(50, 100)
            green = np.random.randint(45, 60)
        elif period == 'off-peak':
            density = np.random.randint(20, 70)
            wait = np.random.randint(10, 40)
            flow = np.random.randint(20, 40)
            queue = np.random.randint(5, 20)
            emergency = np.random.choice([0, 1], p=[0.98, 0.02])
            opp_density = np.random.randint(20, 70)
            green = np.random.randint(20, 45)
        else:  # nighttime
            density = np.random.randint(0, 20)
            wait = np.random.randint(0, 15)
            flow = np.random.randint(0, 20)
            queue = np.random.randint(0, 10)
            emergency = np.random.choice([0, 1], p=[0.99, 0.01])
            opp_density = np.random.randint(0, 20)
            green = np.random.randint(10, 20)
        data['vehicle_density'].append(density)
        data['waiting_time'].append(wait)
        data['flow_rate'].append(flow)
        data['queue_length'].append(queue)
        data['emergency_vehicle'].append(emergency)
        data['opposing_density'].append(opp_density)
        data['green_duration'].append(green)
    return pd.DataFrame(data)

data = generate_realistic_traffic_data(n_samples=10000)
data.to_csv('traffic_data_10000.csv', index=False)
print("Dataset saved to 'traffic_data_10000.csv'")

# Step 2: Load Dataset from CSV

In [ ]:
try:
    data = pd.read_csv('traffic_data_10000.csv')
    print(f"Loaded {len(data)} samples from 'traffic_data_10000.csv'")
except FileNotFoundError:
    print("Error: 'traffic_data_10000.csv' not found. Please ensure the file exists.")
    exit(1)

# Step 3: Define Fuzzy Variables and Membership Functions

In [ ]:
vehicle_density = ctrl.Antecedent(np.arange(0, 101, 1), 'vehicle_density')
vehicle_density['low'] = fuzz.trimf(vehicle_density.universe, [0, 0, 50])
vehicle_density['medium'] = fuzz.trimf(vehicle_density.universe, [25, 50, 75])
vehicle_density['high'] = fuzz.trimf(vehicle_density.universe, [50, 100, 100])

waiting_time = ctrl.Antecedent(np.arange(0, 61, 1), 'waiting_time')
waiting_time['short'] = fuzz.trimf(waiting_time.universe, [0, 0, 30])
waiting_time['medium'] = fuzz.trimf(waiting_time.universe, [15, 30, 45])
waiting_time['long'] = fuzz.trimf(waiting_time.universe, [30, 60, 60])

flow_rate = ctrl.Antecedent(np.arange(0, 61, 1), 'flow_rate')
flow_rate['low'] = fuzz.trimf(flow_rate.universe, [0, 0, 25])
flow_rate['medium'] = fuzz.trimf(flow_rate.universe, [15, 35, 55])
flow_rate['high'] = fuzz.trimf(flow_rate.universe, [45, 60, 60])

queue_length = ctrl.Antecedent(np.arange(0, 31, 1), 'queue_length')
queue_length['short'] = fuzz.trimf(queue_length.universe, [0, 0, 12])
queue_length['medium'] = fuzz.trimf(queue_length.universe, [8, 18, 28])
queue_length['long'] = fuzz.trimf(queue_length.universe, [22, 30, 30])

emergency_vehicle = ctrl.Antecedent(np.arange(0, 2, 1), 'emergency_vehicle')
emergency_vehicle['absent'] = fuzz.trimf(emergency_vehicle.universe, [0, 0, 0])
emergency_vehicle['present'] = fuzz.trimf(emergency_vehicle.universe, [1, 1, 1])

opposing_density = ctrl.Antecedent(np.arange(0, 101, 1), 'opposing_density')
opposing_density['low'] = fuzz.trimf(opposing_density.universe, [0, 0, 50])
opposing_density['medium'] = fuzz.trimf(opposing_density.universe, [25, 50, 75])
opposing_density['high'] = fuzz.trimf(opposing_density.universe, [50, 100, 100])

green_duration = ctrl.Consequent(np.arange(10, 61, 1), 'green_duration')
green_duration['short'] = fuzz.trimf(green_duration.universe, [10, 10, 30])
green_duration['medium'] = fuzz.trimf(green_duration.universe, [20, 35, 50])
green_duration['long'] = fuzz.trimf(green_duration.universe, [40, 60, 60])

# Step 4: Define Expanded Fuzzy Rules

In [ ]:
rule1 = ctrl.Rule(vehicle_density['high'] & waiting_time['long'] & flow_rate['high'] & queue_length['long'] & emergency_vehicle['absent'] & opposing_density['low'], green_duration['long'])
rule2 = ctrl.Rule(vehicle_density['medium'] & waiting_time['medium'] & flow_rate['medium'] & queue_length['medium'] & emergency_vehicle['absent'] & opposing_density['medium'], green_duration['medium'])
rule3 = ctrl.Rule(vehicle_density['low'] & waiting_time['short'] & flow_rate['low'] & queue_length['short'] & emergency_vehicle['absent'] & opposing_density['high'], green_duration['short'])
rule4 = ctrl.Rule(emergency_vehicle['present'], green_duration['long'])
rule5 = ctrl.Rule(flow_rate['high'] & queue_length['long'] & opposing_density['low'], green_duration['long'])
rule6 = ctrl.Rule(opposing_density['high'] & vehicle_density['low'], green_duration['short'])
rule7 = ctrl.Rule(vehicle_density['high'] & queue_length['short'] & flow_rate['medium'], green_duration['medium'])
rule8 = ctrl.Rule(waiting_time['long'] & flow_rate['low'] & queue_length['medium'], green_duration['medium'])
rule9 = ctrl.Rule(vehicle_density['low'] & opposing_density['low'] & emergency_vehicle['absent'], green_duration['short'])

# Step 5: Create Fuzzy Control System

In [ ]:
traffic_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9])
traffic_sim = ctrl.ControlSystemSimulation(traffic_ctrl)

# Step 6: Evaluate Performance Across Sample Sizes with Plots

In [ ]:
sample_sizes = [100, 500, 1000, 2500, 5000, 7500, 10000]
performance_data = {
    'sample_size': [],
    'avg_wait_time': [],
    'training_time': []
}

for n_samples in sample_sizes:
    print(f"\nEvaluating performance with {n_samples} samples...")
    subset_data = data.sample(n=n_samples, random_state=42)
    train_data = subset_data.sample(frac=0.8, random_state=42)
    test_data = subset_data.drop(train_data.index)
    
    start_time = time.time()
    X_train = train_data[['vehicle_density', 'waiting_time', 'flow_rate', 'queue_length', 'emergency_vehicle', 'opposing_density']].values
    y_train = train_data['green_duration'].values
    mlp = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)
    mlp.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    wait_times = []
    predicted_green_durations = []
    for i in test_data.index:
        inputs = {
            'vehicle_density': np.clip(test_data.loc[i, 'vehicle_density'], 0, 100),
            'waiting_time': np.clip(test_data.loc[i, 'waiting_time'], 0, 60),
            'flow_rate': np.clip(test_data.loc[i, 'flow_rate'], 0, 60),
            'queue_length': np.clip(test_data.loc[i, 'queue_length'], 0, 30),
            'emergency_vehicle': np.clip(test_data.loc[i, 'emergency_vehicle'], 0, 1),
            'opposing_density': np.clip(test_data.loc[i, 'opposing_density'], 0, 100)
        }
        if any(np.isnan(val) for val in inputs.values()):
            print(f"Warning: Invalid input for sample {i}: {inputs}")
            predicted_green_durations.append(10)
            wait_times.append(test_data.loc[i, 'waiting_time'])
            continue
        
        traffic_sim.input['vehicle_density'] = inputs['vehicle_density']
        traffic_sim.input['waiting_time'] = inputs['waiting_time']
        traffic_sim.input['flow_rate'] = inputs['flow_rate']
        traffic_sim.input['queue_length'] = inputs['queue_length']
        traffic_sim.input['emergency_vehicle'] = inputs['emergency_vehicle']
        traffic_sim.input['opposing_density'] = inputs['opposing_density']
        try:
            traffic_sim.compute()
            fuzzy_green = traffic_sim.output['green_duration']
            nn_green = mlp.predict([[inputs['vehicle_density'], inputs['waiting_time'], 
                                    inputs['flow_rate'], inputs['queue_length'], 
                                    inputs['emergency_vehicle'], inputs['opposing_density']]])[0]
            green_time = 0.7 * fuzzy_green + 0.3 * nn_green
            green_time = np.clip(green_time, 10, 60)
            predicted_green_durations.append(green_time)
            wait_time = max(0, test_data.loc[i, 'waiting_time'] - green_time)
            wait_times.append(wait_time)
        except Exception as e:
            print(f"Error during simulation for sample {i}: {e}")
            print(f"Inputs: {inputs}")
            predicted_green_durations.append(10)
            wait_times.append(test_data.loc[i, 'waiting_time'])
    
    avg_wait_time = np.mean(wait_times)
    performance_data['sample_size'].append(n_samples)
    performance_data['avg_wait_time'].append(avg_wait_time)
    performance_data['training_time'].append(training_time)
    print(f"Sample Size: {n_samples}, Avg Wait Time: {avg_wait_time:.2f} seconds, Training Time: {training_time:.2f} seconds")
    
    plt.figure(figsize=(10, 6))
    plt.scatter(test_data['vehicle_density'], test_data['waiting_time'], c=predicted_green_durations, cmap='viridis')
    plt.colorbar(label='Green Light Duration (seconds)')
    plt.xlabel('Vehicle Density')
    plt.ylabel('Waiting Time (seconds)')
    plt.title(f'Traffic Light Control: Green Light Duration ({n_samples} Samples, Test Set)')
    plt.savefig(f'traffic_plot_{n_samples}.png')
    plt.close()
    print(f"Traffic plot for {n_samples} samples saved to 'traffic_plot_{n_samples}.png'")

# Step 7: Save Performance Metrics to CSV

In [ ]:
performance_df = pd.DataFrame(performance_data)
performance_df.to_csv('performance_metrics.csv', index=False)
print("\nPerformance metrics saved to 'performance_metrics.csv'")

# Step 8: Visualize Performance Metrics

In [ ]:
if not performance_data['sample_size']:
    print("Error: No performance metrics to plot. Check data generation.")
else:
    print("Performance Metrics:", performance_data)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(performance_data['sample_size'], performance_data['avg_wait_time'], marker='o')
    plt.xlabel('Sample Size')
    plt.ylabel('Average Waiting Time (seconds)')
    plt.title('Average Waiting Time vs. Sample Size')
    plt.grid(True)
    plt.subplot(1, 2, 2)
    plt.plot(performance_data['sample_size'], performance_data['training_time'], marker='o', color='orange')
    plt.xlabel('Sample Size')
    plt.ylabel('Training Time (seconds)')
    plt.title('Training Time vs. Sample Size')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('performance_plot.png')
    plt.close()
    print("Performance plot saved to 'performance_plot.png'")

# Step 9: Run Full Evaluation with 10000 Samples

In [ ]:
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

X_train = train_data[['vehicle_density', 'waiting_time', 'flow_rate', 'queue_length', 'emergency_vehicle', 'opposing_density']].values
y_train = train_data['green_duration'].values
mlp = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

wait_times = []
predicted_green_durations = []
test_results = {
    'vehicle_density': [],
    'waiting_time': [],
    'flow_rate': [],
    'queue_length': [],
    'emergency_vehicle': [],
    'opposing_density': [],
    'predicted_green_duration': [],
    'actual_wait_time': []
}

for i in test_data.index:
    inputs = {
        'vehicle_density': np.clip(test_data.loc[i, 'vehicle_density'], 0, 100),
        'waiting_time': np.clip(test_data.loc[i, 'waiting_time'], 0, 60),
        'flow_rate': np.clip(test_data.loc[i, 'flow_rate'], 0, 60),
        'queue_length': np.clip(test_data.loc[i, 'queue_length'], 0, 30),
        'emergency_vehicle': np.clip(test_data.loc[i, 'emergency_vehicle'], 0, 1),
        'opposing_density': np.clip(test_data.loc[i, 'opposing_density'], 0, 100)
    }
    if any(np.isnan(val) for val in inputs.values()):
        print(f"Warning: Invalid input for sample {i}: {inputs}")
        predicted_green_durations.append(10)
        wait_times.append(test_data.loc[i, 'waiting_time'])
        test_results['vehicle_density'].append(test_data.loc[i, 'vehicle_density'])
        test_results['waiting_time'].append(test_data.loc[i, 'waiting_time'])
        test_results['flow_rate'].append(test_data.loc[i, 'flow_rate'])
        test_results['queue_length'].append(test_data.loc[i, 'queue_length'])
        test_results['emergency_vehicle'].append(test_data.loc[i, 'emergency_vehicle'])
        test_results['opposing_density'].append(test_data.loc[i, 'opposing_density'])
        test_results['predicted_green_duration'].append(10)
        test_results['actual_wait_time'].append(test_data.loc[i, 'waiting_time'])
        continue
    
    traffic_sim.input['vehicle_density'] = inputs['vehicle_density']
    traffic_sim.input['waiting_time'] = inputs['waiting_time']
    traffic_sim.input['flow_rate'] = inputs['flow_rate']
    traffic_sim.input['queue_length'] = inputs['queue_length']
    traffic_sim.input['emergency_vehicle'] = inputs['emergency_vehicle']
    traffic_sim.input['opposing_density'] = inputs['opposing_density']
    try:
        traffic_sim.compute()
        fuzzy_green = traffic_sim.output['green_duration']
        nn_green = mlp.predict([[inputs['vehicle_density'], inputs['waiting_time'], 
                                inputs['flow_rate'], inputs['queue_length'], 
                                inputs['emergency_vehicle'], inputs['opposing_density']]])[0]
        green_time = 0.7 * fuzzy_green + 0.3 * nn_green
        green_time = np.clip(green_time, 10, 60)
        wait_time = max(0, test_data.loc[i, 'waiting_time'] - green_time)
        
        predicted_green_durations.append(green_time)
        wait_times.append(wait_time)
        test_results['vehicle_density'].append(inputs['vehicle_density'])
        test_results['waiting_time'].append(inputs['waiting_time'])
        test_results['flow_rate'].append(inputs['flow_rate'])
        test_results['queue_length'].append(inputs['queue_length'])
        test_results['emergency_vehicle'].append(inputs['emergency_vehicle'])
        test_results['opposing_density'].append(inputs['opposing_density'])
        test_results['predicted_green_duration'].append(green_time)
        test_results['actual_wait_time'].append(wait_time)
    except Exception as e:
        print(f"Error during simulation for sample {i}: {e}")
        print(f"Inputs: {inputs}")
        predicted_green_durations.append(10)
        wait_times.append(test_data.loc[i, 'waiting_time'])
        test_results['vehicle_density'].append(inputs['vehicle_density'])
        test_results['waiting_time'].append(inputs['waiting_time'])
        test_results['flow_rate'].append(inputs['flow_rate'])
        test_results['queue_length'].append(inputs['queue_length'])
        test_results['emergency_vehicle'].append(inputs['emergency_vehicle'])
        test_results['opposing_density'].append(inputs['opposing_density'])
        test_results['predicted_green_duration'].append(10)
        test_results['actual_wait_time'].append(test_data.loc[i, 'waiting_time'])

# Step 10: Save Full Test Results to CSV

In [ ]:
test_results_df = pd.DataFrame(test_results)
test_results_df.to_csv('test_results_10000.csv', index=False)
print("Full test set results saved to 'test_results_10000.csv'")

# Step 11: Save Detailed Results for 10000 Samples

In [ ]:
avg_wait_time = np.mean(wait_times)
with open('traffic_results_10000.txt', 'w') as f:
    f.write(f"Average Waiting Time (Test Set): {avg_wait_time:.2f} seconds\n")
    f.write(f"Number of Training Samples: {len(train_data)}\n")
    f.write(f"Number of Test Samples: {len(test_data)}\n")
    f.write("Sample Test Data (Vehicle Density, Waiting Time, Flow Rate, Queue Length, Emergency Vehicle, Opposing Density, Predicted Green Duration):\n")
    for i, idx in enumerate(test_data.index[:10]):
        f.write(f"{test_data.loc[idx, 'vehicle_density']}, {test_data.loc[idx, 'waiting_time']}, "
                f"{test_data.loc[idx, 'flow_rate']}, {test_data.loc[idx, 'queue_length']}, "
                f"{test_data.loc[idx, 'emergency_vehicle']}, {test_data.loc[idx, 'opposing_density']}, "
                f"{predicted_green_durations[i]:.2f}\n")
print("Detailed results saved to 'traffic_results_10000.txt'")

# Step 12: Visualize Test Set for 10000 Samples

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(test_data['vehicle_density'], test_data['waiting_time'], c=predicted_green_durations, cmap='viridis')
plt.colorbar(label='Green Light Duration (seconds)')
plt.xlabel('Vehicle Density')
plt.ylabel('Waiting Time (seconds)')
plt.title('Traffic Light Control: Green Light Duration (10000 Samples, Test Set)')
plt.savefig('traffic_plot_10000.png')
plt.close()
print("Traffic plot saved to 'traffic_plot_10000.png'")

# Step 13: Prepare Example Test Results for Report

In [ ]:
example_test_results = test_results_df.head(2)[[
    'vehicle_density', 'waiting_time', 'flow_rate', 'queue_length', 
    'emergency_vehicle', 'opposing_density', 'predicted_green_duration', 'actual_wait_time'
]].values.tolist()

# Step 14: Generate the Report with Actual Data

In [ ]:
print(f"Type of performance_data: {type(performance_data)}")
print(f"Content of performance_data: {performance_data}")
print(f"Type of avg_wait_time: {type(avg_wait_time)}")
print(f"Content of avg_wait_time: {avg_wait_time}")
print(f"Type of example_test_results: {type(example_test_results)}")
print(f"Content of example_test_results: {example_test_results}")
print(f"Type of generate_report: {type(generate_report)}")
print(f"generate_report module path: {generate_report.__file__}")


if not isinstance(performance_data, dict):
    print("Error: performance_data is not a dictionary. Check for naming conflicts or redefinition.")
    raise ValueError("performance_data is not a dictionary")
if not isinstance(avg_wait_time, (int, float, np.float64)):
    print("Error: avg_wait_time is not a number. Check simulation logic.")
    raise ValueError("avg_wait_time is not a number")
if not isinstance(example_test_results, list):
    print("Error: example_test_results is not a list. Check test results preparation.")
    raise ValueError("example_test_results is not a list")
if not hasattr(generate_report, 'generate_report'):
    print("Error: generate_report module does not have a generate_report function.")
    raise AttributeError("generate_report function not found in module")

try:
    generate_report.generate_report(avg_wait_time, performance_data, example_test_results)
    print("Report generation completed. Check 'traffic_control_report.docx'.")
except Exception as e:
    print(f"Error generating report: {e}")
    traceback.print_exc()
    print("Ensure 'generate_report.py' is in the same directory as this notebook and 'python-docx' is installed.")
    print("Check for files named 'performance_metrics.py', 'perf_metrics.py', or similar in the directory.")
    print("Verify that 'generate_report' is not redefined in the notebook (e.g., as a variable).")